In [ ]:
from random import sample
import json
import base64
import gzip
from io import BytesIO
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from sqlalchemy import select, func
from eyened_orm import (
    ImageInstance,
    Modality,
    Feature,
    Annotation,
    AnnotationData,
    AnnotationType,
    Segmentation,
    Creator
)
from eyened_orm.Segmentation import Datatype, DataRepresentation
from eyened_orm.db import Database

In [ ]:
database = Database('../dev/.env')
session = database.create_session()

In [ ]:
macular_layers = [
        "background",
        "Retinal Nerve Fiber Layer (RNFL)",
        "Ganglion Cell Layer (GCL)",
        "Inner Plexiform Layer (IPL)",
        "Inner Nuclear Layer (INL)",
        "Outer Plexiform Layer (OPL)",
        "Outer Nuclear Layer (ONL)",
        "External Limiting Membrane (ELM)",
        "Myoid Zone (MZ)",
        "Ellipsoid Zone (EZ)",
        "Outer Segments (OS)",
        "Inter Digitation Zone (IDZ)",
        "Retinal Pigment Epithelium (RPE)",
        "Choroid",
        "Other"
]
optic_disc_features = [
        "background",
        "Retinal Nerve Fiber Layer (RNFL)",
        "Bruch's Membrane (BM)",
        "Lamina Cribosa",
        "Vessels",
        "PPA alpha",
        "PPA beta",
        "PPA gamma"
]

In [ ]:
# feature_macular_layers = Feature.from_list(session, "Macular Layers NEW", macular_layers)

In [ ]:
feature_macular_layers = Feature.by_name(session, "Macular Layers NEW")
if feature_macular_layers is None:
    feature_macular_layers = Feature.from_list(session, "Macular Layers NEW", macular_layers)
    session.add(feature_macular_layers)
    session.commit()

feature_optic_disc_features = Feature.by_name(session, "Optic Disc Features NEW")
if feature_optic_disc_features is None:
    feature_optic_disc_features = Feature.from_list(session, "Optic Disc Features NEW", optic_disc_features)
    session.add(feature_optic_disc_features)
    session.commit()

In [ ]:
def get_annotations_with_annotation_type(annotation_type_ids, where=None):
    #
    query = (
        select(Annotation, ImageInstance)
        # .join_from(Annotation, AnnotationData, isouter=True)
        .join_from(Annotation, ImageInstance, isouter=True)
        .join_from(Annotation, Creator)
        .where(
            ~Annotation.Inactive & 
            (Annotation.AnnotationTypeID.in_(annotation_type_ids)) &
            (Annotation.CreatorID != 1) &
            (Creator.IsHuman)
        )
    )
    
    if where is not None:
        query = query.where(where)
    
    all_annots = session.execute(
        query
        .order_by(func.random())
        .limit(5)
    ).all()
    return all_annots

In [ ]:
annots =get_annotations_with_annotation_type([16])

In [ ]:
for annot, im in annots:
    print(annot.AnnotationData[0].path)

In [ ]:
# BASIC ANNOTATIONS
def open_data(dpath, db_res=None):
    assert db_res is not None, "db_res is required for .gz files"
    with gzip.open(dpath, 'rb') as f:
        im = np.frombuffer(f.read(), dtype=np.uint8)
        im = im.reshape(db_res) # HWD
        # transpose to DHW
        # im = im.transpose(2,0,1)
    return im # DHW

def convert_annotations(annotation_type_id, where=None):
    elems = get_annotations_with_annotation_type([annotation_type_id], where=where)
    annotations = []
    segmentations = []
    # ignore Vessel masks here. They will be inserted with the Artery/Vein annotations
    for annot, image_instance in elems:
        if len(annot.AnnotationData) == 0:
            print('Found no annotation data for ', annot.AnnotationID)
            continue

        if len(annot.AnnotationData) > 1:
            print('Found multiple annotation data for ', annot.AnnotationID)
            continue
        annot_data = annot.AnnotationData[0]
        res_db = (image_instance.NrOfFrames, image_instance.Rows_y, image_instance.Columns_x)

        try:
            im = open_data(annot_data.path, res_db)
        except Exception as e:
            raise RuntimeError(f'Error opening {annot_data.path}: {e}') from e
        
        if len(im.shape) != 3:
            raise RuntimeError(f'Found shape {im.shape} for {annot_data.path}')
        
        print(im.shape, res_db)

        depth, height, width = im.shape
        # map from old features to new composite features
        if annot.FeatureID == 45:
            feature_id = feature_macular_layers.FeatureID
        elif annot.FeatureID == 129:
            feature_id = feature_optic_disc_features.FeatureID
        else:
            raise RuntimeError(f'Unknown feature ID: {annot.FeatureID}')
        segmentation = Segmentation(
            Depth=depth,
            Height=height,
            Width=width,
            SparseAxis=0,
            ScanIndices=None,
            ImageProjectionMatrix=None,
            DataRepresentation=DataRepresentation.MultiClass,
            DataType=Datatype.R8UI,
            ImageInstanceID=image_instance.ImageInstanceID,
            CreatorID=annot.CreatorID,
            FeatureID = feature_id
        )

        session.add(segmentation)
        session.flush([segmentation])

        segmentation.write_data(im)

        segmentations.append(segmentation)
        annotations.append(annot)

    session.commit()
    return annotations, segmentations

In [ ]:
annotations, segmentations  = convert_annotations(16)

In [ ]:
for annot, seg in zip(annotations, segmentations):
    print(annot.AnnotationID, seg.SegmentationID, seg.Feature.FeatureName, seg.ImageInstanceID)